In [1]:
from skimage import io
from skimage import transform
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import os
import warnings 
warnings.filterwarnings('ignore')

In [6]:
pic_array_gray = io.imread('../data/flower_data/train/daisy/100080576_f52e8ee070_n.jpg',as_gray=True)
# plt.imshow(pic_array_gray)
# pic_array_gray = transform.resize(pic_array_gray,(300,300))
pic_flatten = pic_array_gray.flatten()
# pic_flatten.shape
label = {
    'daisy':0,
    'roses':1
}
pic_array_gray.shape

(263, 320)

In [8]:
def make_data(path):
    piclist = os.listdir(path)
    pic_path = [path + pic for pic in piclist]
    list_pic = []
    for pic in pic_path:
        pic_flatten = transform.resize(io.imread(pic,as_gray=True),(300,300)).flatten()
        list_pic.append(pic_flatten)
    return list_pic

In [12]:
# 小雏菊  ###start####
path = '../data/flower_data/train/daisy/'
daisy_data = make_data(path=path)
daisy_data = np.array(daisy_data)
daisy_data

array([[0.5289966 , 0.54199084, 0.55855264, ..., 0.50456802, 0.50452771,
        0.50452771],
       [0.85024906, 0.81535252, 0.83513135, ..., 0.6843597 , 0.68391224,
        0.61109268],
       [0.51829202, 0.49667348, 0.49992162, ..., 0.12702959, 0.12157098,
        0.12721457],
       ...,
       [0.43804784, 0.42773541, 0.41478304, ..., 0.26575751, 0.26829815,
        0.26177598],
       [0.30611508, 0.30573945, 0.31680883, ..., 0.81905683, 0.86788181,
        0.88403357],
       [0.47916013, 0.48159715, 0.48890824, ..., 0.39796247, 0.36564494,
        0.35487243]])

In [14]:
# 小雏菊 分类（y = 0）
daisy_labels = np.ones((daisy_data.shape[0],1))
daisy_data = np.concatenate([daisy_data,daisy_labels],axis=1)
daisy_labels.shape
# 小雏菊  ###end####

(633, 1)

In [54]:
# 玫瑰
path = '../data/flower_data/flower_data/train/roses/'
roses_data = make_data(path=path)
roses_data = np.array(roses_data)
roses_labels = np.zeros((roses_data.shape[0],1))
roses_data = np.concatenate([roses_data,roses_labels],axis=1)

In [56]:
data = np.concatenate([daisy_data,roses_data],axis=0)
data.shape

(1275, 90001)

In [1]:
# plt.imshow(data[-1][:-1].reshape(300,300),cmap='gray')

In [2]:
# plt.imshow(data[0][:-1].reshape(300,300),cmap='gray')

In [65]:
# res = pd.DataFrame(data=data)
# res.to_csv('flower.csv')

In [85]:
from sklearn.linear_model import LogisticRegression # 逻辑回归
from sklearn.model_selection import KFold           # K-折交叉验证法
from sklearn.metrics import accuracy_score

In [5]:
def kfold(x,y):
    kfold = KFold(n_splits=5,shuffle=True)
    # 超参数池（L2)
    c_param = [0.1,0.2,0.4,0.6,0.8,1.0]
    # 遍历参数
    acc_info_list = []
    for c in c_param:
        acc_list = []
        print('~~~~~~~~~~~~~~~~~~~')
        print('正则惩罚项力度：',c)
        print('~~~~~~~~~~~~~~~~~~~')
        for train_index,test_index in kfold.split(x):
            x_train = x[train_index]
            y_train = y[train_index]
            x_test = x[test_index]
            y_test = y[test_index]
            lr = LogisticRegression(C=c)
            lr.fit(x_train,y_train)
            # 预测 
            prediction = lr.predict(x_test)
            acc = accuracy_score(y_true=y_test,y_pred=prediction)
            print(acc)
            acc_list.append(acc)      
        acc_info_list.append(acc_list)
    return acc_info_list

In [3]:
# x = data[:,:-1]
# y = data[:,-1]
# res = kfold(x,y)

In [121]:
res_df = pd.DataFrame(res,columns=['k-1','k-2','k-3','k-4','k-5'])
res_df['c'] = np.array([0.1,0.2,0.4,0.6,0.8,1.0])
res_df
max_index = res_df.mean().argmax()
best_c = res_df.iloc[max_index,:]['c']
best_c

0.2

In [122]:
x = data[:,:-1]
y = data[:,-1]
lr = LogisticRegression(C=best_c)
lr.fit(x,y)

LogisticRegression(C=0.2)

In [124]:
import joblib
# 模型保存
joblib.dump(lr,'flower.pkl')

['flower.pkl']